In [1]:
from imports import *
import pyspark
import pydataset



1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 09:31:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
pandas_df = pd.DataFrame(dict(lang=np.random.choice(['Python', 'SQL', 'R', 'Julia', 'Solidity'], 20),n=np.arange(20)))

pandas_df


,lang,n
0,Julia,0
1,Solidity,1
2,Python,2
3,SQL,3
4,SQL,4
5,R,5
6,SQL,6
7,R,7
8,Julia,8
9,Julia,9


In [5]:
df = spark.createDataFrame(pandas_df)
df

DataFrame[lang: string, n: bigint]

In [113]:
df.printSchema()

root
 |-- lang: string (nullable = true)
 |-- n: long (nullable = true)



In [124]:
print('DataFrame shape: ', df.count(), 'x', len(df.columns))

DataFrame shape:  20 x 2


In [16]:
df.show(5)

+--------+---+
|    lang|  n|
+--------+---+
|   Julia|  0|
|Solidity|  1|
|  Python|  2|
|     SQL|  3|
|     SQL|  4|
+--------+---+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:

"The 1999 audi a4 has a 4 cylinder engine."

for each vehicle.

b. Transform the trans column so that it only contains either manual or auto.

In [22]:
mpg = spark.createDataFrame(pydataset.data('mpg'))

mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [33]:
from builtins import *
from pyspark.sql.functions import *

mpg.select(
    concat(
        lit('The '), 
        'year', 
        lit(' '), 
        'manufacturer', 
        lit(' '), 'model', 
        lit(' has a '), 
        'cyl', 
        lit(' cylinder engine.')
        )
    .alias('engine statement')
    ).show(truncate=False)

+--------------------------------------------------------------+
|engine statement                                              |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

In [60]:
mpg.select('trans', regexp_extract('trans', r'\w{4,6}', 0).alias('simple_trans')).show(5)

+----------+------------+
|     trans|simple_trans|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
+----------+------------+
only showing top 5 rows



3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [41]:
tips = spark.createDataFrame(pydataset.data('tips'))

tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [91]:
tips.filter(tips.smoker == 'Yes').count() / tips.select(tips.smoker == 'Yes').count()

0.38114754098360654

In [101]:
tips = tips.select('*', (tips.tip / tips.total_bill).alias('tip_pct'))

tips.show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|            tip_pct|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



In [103]:
tips.groupBy('sex', 'smoker').agg(avg(tips.tip_pct)).show()

+------+------+-------------------+
|   sex|smoker|       avg(tip_pct)|
+------+------+-------------------+
|  Male|    No| 0.1606687151291298|
|Female|    No| 0.1569209707691836|
|  Male|   Yes|0.15277117520248512|
|Female|   Yes|0.18215035269941032|
+------+------+-------------------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [104]:
from vega_datasets import data

seattle_wx = data.seattle_weather().assign(date= lambda df: df.date.astype(str))

seattle_wx = spark.createDataFrame(seattle_wx)

seattle_wx.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [112]:
print(f'12.8 degrees Celsius is {12.8*(9/5) + 32:.2f} fahrenheit.')

12.8 degrees Celsius is 55.04 fahrenheit.


In [126]:
seattle_wx = seattle_wx.select(
    'date', 
    'precipitation', 
    'temp_max', 
    (round(seattle_wx.temp_max*1.8 + 32,2)).alias('temp_max_fahrenheit'), 
    'temp_min', 
    (round(seattle_wx.temp_min*1.8 + 32,2)).alias('temp_min_fahrenheit'), 
    'wind', 
    'weather'
    )

seattle_wx.show(10)

+----------+-------------+--------+-------------------+--------+-------------------+----+-------+
|      date|precipitation|temp_max|temp_max_fahrenheit|temp_min|temp_min_fahrenheit|wind|weather|
+----------+-------------+--------+-------------------+--------+-------------------+----+-------+
|2012-01-01|          0.0|    12.8|              55.04|     5.0|               41.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|              51.08|     2.8|              37.04| 4.5|   rain|
|2012-01-03|          0.8|    11.7|              53.06|     7.2|              44.96| 2.3|   rain|
|2012-01-04|         20.3|    12.2|              53.96|     5.6|              42.08| 4.7|   rain|
|2012-01-05|          1.3|     8.9|              48.02|     2.8|              37.04| 6.1|   rain|
|2012-01-06|          2.5|     4.4|              39.92|     2.2|              35.96| 2.2|   rain|
|2012-01-07|          0.0|     7.2|              44.96|     2.8|              37.04| 2.3|   rain|
|2012-01-08|        

In [133]:
seattle_wx = seattle_wx.select('*', month('date').alias('month'), year('date').alias('year'))

seattle_wx.show(5)

+----------+-------------+--------+-------------------+--------+-------------------+----+-------+-----+----+
|      date|precipitation|temp_max|temp_max_fahrenheit|temp_min|temp_min_fahrenheit|wind|weather|month|year|
+----------+-------------+--------+-------------------+--------+-------------------+----+-------+-----+----+
|2012-01-01|          0.0|    12.8|              55.04|     5.0|               41.0| 4.7|drizzle|    1|2012|
|2012-01-02|         10.9|    10.6|              51.08|     2.8|              37.04| 4.5|   rain|    1|2012|
|2012-01-03|          0.8|    11.7|              53.06|     7.2|              44.96| 2.3|   rain|    1|2012|
|2012-01-04|         20.3|    12.2|              53.96|     5.6|              42.08| 4.7|   rain|    1|2012|
|2012-01-05|          1.3|     8.9|              48.02|     2.8|              37.04| 6.1|   rain|    1|2012|
+----------+-------------+--------+-------------------+--------+-------------------+----+-------+-----+----+
only showing top 5 

In [139]:
seattle_wx.groupBy('month').agg(sum('precipitation').alias('total_monthly_precip'))\
.groupBy('month').agg(mean('total_monthly_precip').alias('avg_monthly_precip'))\
.sort(col('avg_monthly_precip').desc()).show()

+-----+------------------+
|month|avg_monthly_precip|
+-----+------------------+
|   11|             642.5|
|   12| 622.7000000000002|
|    3|             606.2|
|   10|             503.4|
|    1|465.99999999999994|
|    2|             422.0|
|    4|             375.4|
|    9|235.49999999999997|
|    5|             207.5|
|    8|             163.7|
|    6|             132.9|
|    7|              48.2|
+-----+------------------+



22/05/19 23:51:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 302505 ms exceeds timeout 120000 ms
22/05/19 23:51:28 WARN SparkContext: Killing executors is not supported by current scheduler.
